This notebook is an experiment with the [NeuralVerification](https://github.com/sisl/NeuralVerification.jl) package (that we shorten to `NV` here) and decomposition methods available in [LazySets](https://github.com/JuliaReach/LazySets.jl/).

In [31]:
using Revise, NeuralVerification, LazySets
using LazySets.Approximations

const NV = NeuralVerification

NeuralVerification

In `NV`, [neural networks](https://en.wikipedia.org/wiki/Neural_network#/media/File:Neural_network_example.svg) are represented as a [vector of layers](https://github.com/sisl/NeuralVerification.jl/blob/master/src/utils/network.jl#L40), where a `Layer` consists of the weights matrix, the bias (an affine translation) and the [activation function](https://en.wikipedia.org/wiki/Activation_function).


```julia
struct Layer{F<:ActivationFunction, N<:Number}
    weights::Matrix{N}
    bias::Vector{N}
    activation::F
end

struct Network
    layers::Vector{Layer} # layers includes output layer
end
```

Now we will work with one "small" examples in [NeuralVerification/examples/networks/](https://github.com/sisl/NeuralVerification.jl/tree/master/examples).

In [113]:
networks_folder = "/Users/forets/.julia/dev/NeuralVerification/examples/networks/"

#model = "cartpole_nnet.nnet" # 4 layers, 16x4 dims
model = "ACASXU_run2a_4_5_batch_2000.nnet" # 7 layers, 50x5 dims

nnet = read_nnet(networks_folder * model);

The number of layers in this neural network as well as the number of nodes in each layer can be obtained as follows.

In [114]:
L = nnet.layers
length(L)

7

The first two layers have two nodes each and the last layer (the output layer) has one node.

In [115]:
NV.n_nodes.(L)

7-element Array{Int64,1}:
 50
 50
 50
 50
 50
 50
  5

In [116]:
dump(L[1])

NeuralVerification.Layer{NeuralVerification.ReLU,Float64}
  weights: Array{Float64}((50, 5)) [0.0553801 0.190198 … -1.32009 -0.704992; -0.010481 -0.118571 … -0.736114 -1.00878; … ; -0.0514028 -1.17772 … 0.382633 -0.500615; -0.936931 -0.203562 … -0.20136 0.320543]
  bias: Array{Float64}((50,)) [-0.557027, -0.317043, 0.138104, 0.189978, 0.188149, 0.0552362, -0.06767, -0.260805, -0.267183, -0.348518  …  0.180223, -0.408499, -0.429542, -0.0201813, 0.0710165, 0.0685803, -0.303011, -0.0773268, 0.0294964, -0.20074]
  activation: NeuralVerification.ReLU NeuralVerification.ReLU()


We can directly see the weights matrix and the bias fields of the first layer:

In [127]:
L[1].weights

50×5 Array{Float64,2}:
  0.0553801    0.190198    -0.0377036   -1.32009      -0.704992  
 -0.010481    -0.118571     0.153946    -0.736114     -1.00878   
  0.482564     0.0565022    0.0443098    0.0600691     0.0020876 
 -0.0778212   -0.0577902   -0.153575     0.0605465     0.450204  
  0.0143941   -0.25066     -0.132743     0.533158     -0.0781143 
 -0.00869665  -0.447464    -0.0517125   -0.0990526     0.204336  
  0.010042    -0.00129605  -0.00514204  -0.0715019    -0.0436908 
 -0.0696566    0.215441    -0.00483027   0.155244     -0.858598  
 -0.0412334    0.845604     0.238394     0.166022     -0.0812237 
  0.386318    -0.0315554    0.00926705  -0.369564     -0.731116  
 -0.727862    -0.136911     0.0147124   -0.76237      -0.181072  
  0.00181044  -0.131803     0.0962716    0.175322     -0.863416  
  0.0213996    0.126362     0.0304863   -1.13285       1.17788   
  ⋮                                                              
 -0.00247847  -0.00046304   0.00565075  -0.000137539 

In [128]:
L[1].bias

50-element Array{Float64,1}:
 -0.557027 
 -0.317043 
  0.138104 
  0.189978 
  0.188149 
  0.0552362
 -0.06767  
 -0.260805 
 -0.267183 
 -0.348518 
 -0.614078 
  0.217921 
 -0.156318 
  ⋮        
 -0.0137361
 -0.652247 
  0.180223 
 -0.408499 
 -0.429542 
 -0.0201813
  0.0710165
  0.0685803
 -0.303011 
 -0.0773268
  0.0294964
 -0.20074  

An input set for ACAS is defined in the `test/runtime3.jl` file so we use it:

In [130]:
center = [0.40143256,  0.30570418, -0.49920412,  0.52838383,  0.4]
radius = [0.0015, 0.0015, 0.0015, 0.0015, 0.0015]
H0 = Hyperrectangle(center, radius)
dim(H)

5

Let $X_1$ be the set obtained after we apply the first layer, $X_1 = A_1 H_0 \oplus b_1$.

In [133]:
A1 = L[1].weights
b1 = L[1].bias
X1 = A1 * H0 ⊕ b1

Translation{Float64,Array{Float64,1},LinearMap{Float64,Hyperrectangle{Float64},Float64,Array{Float64,2}}}(LinearMap{Float64,Hyperrectangle{Float64},Float64,Array{Float64,2}}([0.0553801 0.190198 … -1.32009 -0.704992; -0.010481 -0.118571 … -0.736114 -1.00878; … ; -0.0514028 -1.17772 … 0.382633 -0.500615; -0.936931 -0.203562 … -0.20136 0.320543], Hyperrectangle{Float64}([0.401433, 0.305704, -0.499204, 0.528384, 0.4], [0.0015, 0.0015, 0.0015, 0.0015, 0.0015])), [-0.557027, -0.317043, 0.138104, 0.189978, 0.188149, 0.0552362, -0.06767, -0.260805, -0.267183, -0.348518  …  0.180223, -0.408499, -0.429542, -0.0201813, 0.0710165, 0.0685803, -0.303011, -0.0773268, 0.0294964, -0.20074])

Note here that $X_1$ is of dimension 50:

In [134]:
dim(X1)

50

Let's consider an element from $X_1$ and apply the rectification operation:

In [138]:
an_element(X1)[1:10] # showing first ten entries only

10-element Array{Float64,1}:
 -1.4373405187421722  
 -1.22681127708228    
  0.359548807009313   
  0.429809995050787   
  0.43403164918939596 
 -0.029834832776401976
 -0.1167248623560412  
 -0.4819057030262836  
 -0.08900415865380401 
 -0.695428185257858   

In [142]:
v = LazySets.rectify(an_element(X1))
v[1:10]

10-element Array{Float64,1}:
 0.0                
 0.0                
 0.359548807009313  
 0.429809995050787  
 0.43403164918939596
 0.0                
 0.0                
 0.0                
 0.0                
 0.0                

In [147]:
count(!iszero, v) # number of elements which are not zero

13

In [151]:
# next layer
A2 = L[1].weights
b2 = L[1].bias
X2(Y) = A2 * (Y) ⊕ b2

X2 (generic function with 1 method)

In [153]:
size(A2)

(50, 5)